<a href="https://colab.research.google.com/github/Aditya100300/LLMs_from_scratch/blob/main/Chapter_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Installation of Dependencies

# We install multiple libraries in one cell:
# - unstructured (for reading text-based docs, e.g. PDFs, etc.)
# - sentence_transformers (for local embedding models if you want)
# - openai (only if you want to do OpenAI calls, though not required)
# - plotly (just in case we want to do some charting)
# - langchain (the main library for retrieval-based LLM stuff)
# - tiktoken (helps handle token counting for LLMs)
# - matplotlib (common Python library for plotting)
# - And more advanced (gradio, pdf2image, wandb, chromadb, faiss-gpu, etc.)

!pip install sentence_transformers openai unstructured
!pip install plotly
!pip install langchain
!pip install tiktoken
!pip install matplotlib

# This line uses %pip to quietly install an extended set of libs (like chromadb, faiss, etc.)
# If you see many lines of output, it's just installing packages. Should be fine.
%pip install -Uqqq rich openai tiktoken wandb langchain unstructured tabulate pdf2image chromadb gradio faiss-gpu

# Additional library for text link detection, if needed
!pip install linkify-it-py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.9 MB/s eta 0:00:00
   

In [2]:
# Cell 2: Imports and Basic Setup

import os, sys
import pandas as pd
import numpy as np
from pathlib import Path
import torch
import csv

# Because CSV lines can be large, let's raise the field_size_limit
csv.field_size_limit(sys.maxsize)

# Optional: If you want to store or type your OpenAI key for certain calls
# from getpass import getpass
# openai_key = getpass("Enter your OpenAI API key...")

print("Libraries imported successfully!")


Libraries imported successfully!


In [3]:
# Cell 3: Basic Example of Sentence Transformers + BERT

# This step is optional if you want to see how to do local embedding with 'bert-base-uncased'
# Usually we prefer 'all-MiniLM-L6-v2' or 'all-mpnet-base-v2' from sentence_transformers

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

text = "This is a short example text to see if BERT encodes it."
encoded = tokenizer.encode(text, return_tensors='pt')

with torch.no_grad():
    output = model(encoded)
    # We can get the average (mean) of last_hidden_state as a rough sentence embedding
    sentence_embedding = output.last_hidden_state.mean(dim=1)

print("Embedding shape:", sentence_embedding.shape)
print("Done local BERT example!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Embedding shape: torch.Size([1, 768])
Done local BERT example!


In [4]:
# Cell 4: A sample text to demonstrate chunking or summarizing.
# You can load from a local file or any text source. We'll store it in a variable.

example_text = '''I would like to get your thoughts on the bond yield increase this week.
I'm not worried about the market downturn, but the sudden increase in yields.
On 2/16, the 10-year bond yields increased by almost 9%, and on 2/19, the yield increased by 5%.
Some experts recall the 'taper tantrum' of 2013, where bond prices crashed after the Fed
announced it would begin tapering quantitative easing.
Others see it as a normal feature of an economic recovery.
'''

with open('example_text.txt', 'w') as f:
    f.write(example_text)

print("Sample text saved to example_text.txt")


Sample text saved to example_text.txt


In [6]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:00


In [7]:
# Cell 5: Using LangChain's text loader + text splitter

# If we want to load a text file from disk, we can do:
from langchain.document_loaders import TextLoader
loader = TextLoader("example_text.txt")

# We'll load the text as a Document
docs = loader.load()
print(f"We have {len(docs)} document(s). First doc length in chars:", len(docs[0].page_content))

# Now let's chunk it using a text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
splits = text_splitter.split_documents(docs)
print(f"Number of chunked splits: {len(splits)}")

print("First chunk:\n", splits[0].page_content)


We have 1 document(s). First doc length in chars: 456
Number of chunked splits: 6
First chunk:
 I would like to get your thoughts on the bond yield increase this week.


In [10]:
# Cell 6: Creating a Vector Store with an open-source embedding (like 'all-MiniLM-L6-v2')
!pip install chromadb
from langchain.embeddings import HuggingFaceEmbeddings
# We'll pick a local model name (this is quite good for small tasks)
model_name = "sentence-transformers/all-mpnet-base-v2"

# This can be large, but let's do it.
local_embedder = HuggingFaceEmbeddings(model_name=model_name)

# Now let's create a Chroma vector store from these splits
from langchain.vectorstores import Chroma

db = Chroma.from_documents(splits, embedding=local_embedder)
retriever = db.as_retriever()

print("Vector store created with local embeddings!")


  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━

In [11]:
# Cell 7: Running a quick retrieval-based Q&A with a local or open-source LLM

# For a small open-source LLM, we might use the new 'llama-2' from Hugging Face, or 'OpenAssistant'.
# But for demonstration, let's just do a "fake" or "prompt" approach, or skip the generative step.

query = "Why did bond yields spike in 2013, and is it relevant to 2023?"
docs = retriever.get_relevant_documents(query)
print(f"Top doc chunks returned: {len(docs)}")

for i,doc in enumerate(docs):
    print(f"\n--- Chunk {i+1} (score unknown) ---\n{doc.page_content}")


<ipython-input-11-f2bd44f7e442>:7: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


Top doc chunks returned: 4

--- Chunk 1 (score unknown) ---
I'm not worried about the market downturn, but the sudden increase in yields.

--- Chunk 2 (score unknown) ---
I would like to get your thoughts on the bond yield increase this week.

--- Chunk 3 (score unknown) ---
Some experts recall the 'taper tantrum' of 2013, where bond prices crashed after the Fed

--- Chunk 4 (score unknown) ---
On 2/16, the 10-year bond yields increased by almost 9%, and on 2/19, the yield increased by 5%.


In [13]:
# Cell 8: Optionally, if you do want to use an open source LLM for generation, let's say ChatGLM2 or Dolly
# We'll show how to do it with "OpenAIChat" if you have an API key, but the logic is the same for huggingface_hub.

import os
os.environ["OPENAI_API_KEY"] = "YOUR_KEY_HERE"

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
# We'll create a simple chain
chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.0),
    chain_type="stuff",
    retriever=retriever
)

result = chain.run(query)
print(result)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [14]:
# Cell 9: Summarizing the entire text with a chain

# Another example: Summarize the entire text with a "MapReduce" chain
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.0)

# If we want to do a naive approach with all docs:
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = summary_chain.run(docs)
print(summary)


<ipython-input-14-50b050b168b9>:6: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.0)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [16]:
!pip install gradio

  Using cached gradio-5.23.3-py3-none-any.whl.metadata (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 24.1.0
    Uninstalling aiofiles-24.1.0:
      Successfully uninstalled aiofiles-24.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-client 0.32.1 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.


In [17]:
# Cell 10: Provide a Gradio UI for user queries (all open-source)

import gradio as gr

def answer_question(query):
    # 1) We do a retrieval
    # 2) Summarize or produce answer
    docs = retriever.get_relevant_documents(query)
    if len(docs)==0:
        return "No relevant chunks found. Not sure."

    # For demonstration, let's just combine them:
    combined_text = "\n".join(d.page_content for d in docs)

    # We'll do a naive LLM call with ChatOpenAI or skip if no key
    llm = ChatOpenAI(temperature=0.0)
    prompt = f"""
    You are a helpful Q&A system.
    The user asked: {query}
    We have these relevant text chunks:
    {combined_text}
    Provide a short answer from the text. If not enough info, say "I don't know."
    """
    resp = llm.predict(prompt)
    return resp

demo = gr.Interface(
    fn=answer_question,
    inputs="text",
    outputs="text",
    title="Open-source RAG Demo",
    description="Ask a question about bond yields or other text content loaded."
)

demo.launch(debug=False, share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30e5ab67475101e324.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Summary:

Installing open-source libraries (LangChain, unstructured, Hugging Face, etc.) so all code can run in Colab with no proprietary dependencies.

Demonstrate Local Embeddings via the sentence_transformers approach or the brand-new HuggingFaceEmbeddings.

Building a Chroma vector store for storing chunked doc embeddings.

A Retriever to get relevant chunks from user queries.

Optionally using an open-source LLM for final generation – or if you have an OpenAI key, we show how to call GPT.

A Gradio interface to unify the pipeline for easy user queries.